# Preamble

## Variables

In [1]:
DATA_AMOUNT = [10, 100]
TRAIN_SPLIT = 0.8

## Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import get_text
import ceaser

## Creating Data

In [4]:
trainData, testData = get_text.initialise(ceaser.encrypt, *DATA_AMOUNT, TRAIN_SPLIT, stream=False)

Using the latest cached version of the dataset since manu/project_gutenberg couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\Tristan\.cache\huggingface\datasets\manu___project_gutenberg\default\0.0.0\164853d214065df26a630ee1ab91a0c39e461caf (last modified on Thu Sep  4 19:18:40 2025).


Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]